| **Model**                | **Pages per Dollar** | **Performance on MTEB Eval** | **Max Input** |
|--------------------------|----------------------|------------------------------|---------------|
| text-embedding-3-small    | 62,500               | 62.3%                        | 8191          |
| text-embedding-3-large    | 9,615                | 64.6%                        | 8191          |
| text-embedding-ada-002    | 12,500               | 61.0%                        | 8191          |


In [1]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00


In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [25]:
text = "우주의 기원은 어디인가요?"

# Query Embedding

In [10]:
query_result = embeddings.embed_query(text)

In [15]:
query_result[:5]

[0.0487622432410717,
 -0.005244841333478689,
 0.03377220407128334,
 0.027035608887672424,
 -0.005228111054748297]

# Document Embedding

In [13]:
text_2 = "심해는 해양의 깊은 곳으로, 빛이 거의 닿지 않는 200m 이하의 영역입니다. 고압, 저온 환경에서 독특한 생물이 서식합니다."

doc_result = embeddings.embed_documents(
    [text_2, text_2, text_2, text_2]
)   # 텍스트를 임베딩하여 문서 벡터를 생성

In [14]:
doc_result[0][:5]

[0.0064157177694141865,
 -9.356773080071434e-05,
 0.006743709556758404,
 0.03315698727965355,
 0.026159828528761864]

# 차원 지정

In [27]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 1024차원의 임베딩을 생성하는 객체를 초기화
scaled_embeddings = OpenAIEmbeddings(model = "text-embedding-3-small", dimensions = 1024)

In [48]:
len(scaled_embeddings.embed_documents([text])[0])

1024


# Similarity Calc.

In [35]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]

In [40]:
embedded_sentences = scaled_embeddings.embed_documents(sentences)

In [41]:
len(embedded_sentences[0])

1024

In [49]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [50]:
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8378] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5044] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1363] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8144] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4792] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5128] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.


# 언어 간 임베딩 차이 줄이기

In [52]:
from langchain_openai import ChatOpenAI
from sklearn.metrics.pairwise import cosine_similarity

# GPT-4o 모델 설정
chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

# GPT-4o를 사용해 문장을 영어로 번역하는 함수
def translate_to_english(sentence):
    response = chat_model.predict_messages(
        [
            {"role": "system", "content": "Translate the given text to English."},
            {"role": "user", "content": sentence},
        ]
    )
    return response.content

# 문장 리스트
sentences = [
    "안녕하세요? 반갑습니다.",
    "안녕하세요? 반갑습니다!",
    "안녕하세요? 만나서 반가워요.",
    "Hi, nice to meet you.",
    "I like to eat apples.",
]

# 문장을 영어로 번역
translated_sentences = [translate_to_english(sentence) for sentence in sentences]

# scaled_embeddings를 적절히 설정하세요
embedded_sentences = scaled_embeddings.embed_documents(translated_sentences)

# 유사도 계산 함수
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

# 유사도 출력
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

<ipython-input-52-69b360b97c52>:9: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model.predict_messages(


[유사도 0.9459] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 1.0000] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.8406] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.2018] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.9459] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.7976] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1878] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.8406] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.2018] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.
